In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

## Setup

- Load data
- Add metadata
- Draw gates
- Gate transfected cells

Result from this section: DataFrame `df` representing transfected cells.

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'/'chris'/'2024.05.28-miR-iFFL-cre'/'export'

plates = pd.DataFrame({
    'data_path': [base_path/'plate01', base_path/'plate02'],
    'yaml_path': [base_path/'plate01_metadata.yaml', base_path/'plate02_metadata.yaml'],
})

output_path = rd.rootdir/'output'/'cre-transfection'
cache_path = output_path/'data.gzip'

metadata_keys = set()
for p in plates['yaml_path'].unique():
    print(p)
    rd.plot.plot_well_metadata(p)
    metadata_keys.update(rd.flow.load_well_metadata(p).keys())
display(metadata_keys)

Channels:

- TagBFP: transfection marker
- eGFP: Cre activity reporter
- mRuby2: Cre-2A-mRuby2 (controller)
- Zombie-NIR: live/dead dye

In [ ]:
# Load data
data = pd.DataFrame()
if cache_path.is_file(): data = pd.read_parquet(cache_path)
else: 
    channel_list = ['mRuby2-A','eGFP-A','TagBFP-A','Zombie-NIR-A','FSC-A','SSC-A',]
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data = data[data[c]>0]
    
    data.dropna(inplace=True)
    data.to_parquet(rd.outfile(cache_path))
display(data)

In [ ]:
# Add metadata for constructs
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx')
data = data.merge(metadata, how='left', on='construct')

In [ ]:
# Gate on transfected cells
gate_transfection = data.loc[(data['construct']=='UT') & (data['has_zombie_dye']==False), 'TagBFP-A'].quantile(0.995)
g = sns.FacetGrid(data=data, hue='cre_amount', row='has_zombie_dye', col='construct', margin_titles=True)
g.map(sns.kdeplot, 'TagBFP-A', fill=False, common_norm=False, log_scale=True)
for name, ax in g.axes_dict.items():
    ax.axvline(gate_transfection, color='black')

In [ ]:
plot_df = data[data['construct']=='UT']
fig, axes = plt.subplots(1,len(channel_list), figsize=(20,4), gridspec_kw=dict(wspace=0.3), sharey=True)
for i,channel in enumerate(channel_list):
    sns.kdeplot(data=plot_df, x=channel, y='Zombie-NIR-A', hue='has_zombie_dye', palette={True: 'purple', False: 'gray'},
                fill=False, levels=7, log_scale=True, legend=False, common_norm=False, ax=axes[i])
    if channel=='TagBFP-A': axes[i].axvline(gate_transfection, color='black')
fig.savefig(output_path/f'joint_channels-vs-zombie.svg')

In [ ]:
df_gated = data[(data['TagBFP-A']>gate_transfection) & (data['has_zombie_dye']==True) & (data['construct']!='UT')]

In [ ]:
grouped = df_gated.groupby(by=['construct','cre_amount'])
stats = grouped[channel_list].agg(sp.stats.gmean).reset_index().dropna()

# Rename columns as 'col_stat'
stats['count'] = grouped['TagBFP-A'].count().reset_index()['TagBFP-A']
stats = stats.merge(metadata, how='left', on='construct')
stats.rename(columns=lambda x: x.replace('-A', '_gmean'), inplace=True)
display(stats)

In [ ]:
mycolors = {
    'RC271': 'black',
    'RC272': 'gray',
    'RC273': 'teal',
    'filler': 'crimson'
}
mymarkers = {
    'RC271': 'X',
    'RC272': 'o',
    'RC273': 'D',
    'filler': 'P'
}

In [ ]:
plot_df = stats
fig, ax = plt.subplots(1,1)

for name, group in plot_df.groupby('construct'):
    sns.scatterplot(data=group, x='cre_amount', y='eGFP_gmean', ax=ax, hue='construct', palette=mycolors)
ax.set(yscale='log')
fig.savefig(output_path/'scatter_cre-amt-vs-reporter.svg')

In [ ]:
plot_df = stats[stats['construct']!='filler']
fig, ax = plt.subplots(1,1)

for name, group in plot_df.groupby('construct'):
    sns.scatterplot(data=group, x='mRuby2_gmean', y='eGFP_gmean', ax=ax, hue='cre_amount', palette='flare', hue_norm=matplotlib.colors.LogNorm(),
                    style='construct', markers=mymarkers, legend=False)
ax.set(yscale='log', xscale='log')
ax.axvline(1e2, color='black', zorder=0)
maxline = stats.loc[stats['mRuby2_gmean']>1e2, 'eGFP_gmean'].mean()
ax.axhline(maxline, color='black', zorder=0)
fig.savefig(output_path/'scatter_cre-level-vs-reporter.svg')

In [ ]:
plot_df = df_gated
g = sns.FacetGrid(data=plot_df, col='construct', hue='cre_amount', palette='flare', )
g.map(sns.kdeplot, 'eGFP-A', hue_norm=matplotlib.colors.LogNorm(),
        fill=False, common_norm=False, log_scale=True)
gate_reporter = data.loc[(data['construct']=='UT') & (data['has_zombie_dye']==False), 'eGFP-A'].quantile(0.995)
for name, ax in g.axes_dict.items():
    ax.axvline(gate_reporter, color='black')
g.figure.savefig(output_path/'hist_cre-reporter.svg')

In [ ]:
plot_df = df_gated
g = sns.FacetGrid(data=plot_df, col='construct', hue='cre_amount', palette='magma',)
g.map(sns.kdeplot, 'mRuby2-A', 'eGFP-A', hue_norm=matplotlib.colors.LogNorm(),
        fill=False, common_norm=False, log_scale=True, levels=4)
g.figure.savefig(output_path/'joint_cre-reporter.svg')

In [ ]:
plot_df = df_gated[(df_gated['construct']=='filler') & (df_gated['eGFP-A']>gate_reporter)]
g = sns.kdeplot(data=plot_df, x='eGFP-A', hue='cre_amount', palette='magma', hue_norm=matplotlib.colors.LogNorm(),
                fill=False, common_norm=False, log_scale=True)

In [ ]:
plot_df = stats[stats['construct']!='filler']
fig, ax = plt.subplots(1,1)

for name, group in plot_df.groupby('construct'):
    sns.scatterplot(data=group, x='count', y='eGFP_gmean', ax=ax, hue='cre_amount', palette='flare', hue_norm=matplotlib.colors.LogNorm(),
                    style='construct', markers=mymarkers, legend=False)
ax.set(yscale='log',)
fig.savefig(output_path/'scatter_count-vs-reporter.svg')

In [ ]:
plot_df = stats[stats['construct']!='filler']
fig, ax = plt.subplots(1,1)

for name, group in plot_df.groupby('construct'):
    sns.scatterplot(data=group, x='count', y='mRuby2_gmean', ax=ax, hue='cre_amount', palette='flare', hue_norm=matplotlib.colors.LogNorm(),
                    style='construct', markers=mymarkers, legend=False)
ax.set(yscale='log',)
fig.savefig(output_path/'scatter_count-vs-cre-level.svg')

In [ ]:
plot_df = stats[stats['construct']!='filler']
fig, ax = plt.subplots(1,1)

for name, group in plot_df.groupby('construct'):
    sns.scatterplot(data=group, x='count', y='mRuby2_gmean', ax=ax, hue='construct', palette=mycolors,
                    style='construct', markers=mymarkers, legend=False)
ax.set(yscale='log',)
fig.savefig(output_path/'scatter_count-vs-cre-level_construct-colors.svg')